# Setup environment

In [1]:
import datetime
from dotenv import load_dotenv, find_dotenv
import os
from typing import List, Dict, Any
import pprint

In [2]:
from IPython.display import display, HTML

In [3]:
# https://pypi.org/project/openai/
import openai
from openai.openai_object import OpenAIObject

In [4]:
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY', 'MISSING_OPENAI_API_KEY')

# Prompt LLM

## Create helpers

In [5]:
def get_completion(prompt: str, model: str, temperature: float, n_completions: int) -> str:
    messages: List[Dict[str, Any]] = [
        {
            "role": "user",
            "content": prompt
        }
    ]
    try:
        response: OpenAIObject = openai.ChatCompletion.create(
            messages=messages,
            model=model,
            temperature=temperature,
            n=n_completions
        ) 
    except Exception as e:
        return f"Error: {e}"

    if not response.choices:
        return "Error: No completions returned"

    return response.choices[0].message.content

## Set prompting params

In [6]:
# https://platform.openai.com/docs/models/model-endpoint-compatibility
completion_model = 'gpt-3.5-turbo'

In [7]:
# https://platform.openai.com/docs/api-reference/completions/create#completions/create-temperature
temperature = 0.01

In [8]:
# https://platform.openai.com/docs/api-reference/completions/create#completions/create-n
n_completions = 1

In [9]:
prompting_params: Dict[str, Any] = {
    "model": completion_model,
    "temperature": temperature,
    "n_completions": n_completions
}

## Start prompting

### Task: Generate a marketing product description from a product fact sheet

source: https://prom.ua/ua/p1824783668-paverbankpowerbank-usb-yusbpoverbank.html

In [10]:
fact_sheet = f"""
OVERVIEW
- Назва: Павербанк Powerbank 2 usb юсб Повербанк зі швидким заряджанням Power bank \
швидке заряджання Power bank для айфона
- Підходить для резервного живлення роутера.
- Оптимальний запас
- Зовнішній акумулятор представлений у стильному варіанті з компактними габаритами для \
спрощення носіння девайса із собою. Попри це, всередині є достатній для багатьох завдань \
запас енергії — 10 000 мА·год. Маючи під рукою такий ресурс, ви завжди зможете відновити \
енерговтрати не тільки свого смартфона та інших ґаджетів, але й допомогти в цьому оточенні — \
наявність роз'ємів дає змогу зробити це.
- Тонкий корпус
- Варто звернути увагу, що зовнішній акумулятор виготовлений у гранично тонкому форм-факторі. \
Корпус має акуратні обтічні грані та лаконічну зовнішність. \
Пластикова оболонка оберігає батарею від пошкоджень і надає їй вишуканого вигляду. \
Полірована поверхня не схильна до появи подряпин, завдяки чому паувербанк завжди зберігає ідеальний вигляд.
- Два роз'єми
- Панель слотів пристрою має 2 порти, які допоможуть приєднати будь-яку техніку для відновлення \
її ресурсів. Вихідні порти формату USB поєднуються з вхідним MicroUSB.
- Підходить для потужних телефонів
- Якщо ви щасливий володар передових моделей смартфонів, зверніть увагу на представлений повербанк. \
Його заряду вистачить, щоб заповнити енергію в таких телефонах мінімум двічі. Так вас ніколи раптово \
не застане батарея з нульовим зарядом і брак зв'язку через брак енергії в техніці.

BOX SET
- Акумулятор 1 шт.
- Кабель USB/microUSB 1 шт.
- Паковання 1 шт.

SPECS
- Ємність 10000 mAh
- USB 2 шт.
- наявність LED-індикатора
- Тип акумулятора Li-Ion

MATERIALS
- Матеріал пластик

COUNTRY OF ORIGIN
- Китай

MISCELANEOUS
- УЦЕНКА  (наявні дрібні подряпини)
"""

In [11]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet}```
"""
response = get_completion(prompt, **prompting_params)
pprint.pprint(response)

('Introducing the Powerbank 2 USB - the ultimate solution for all your '
 'charging needs! This external battery pack is designed to be compact and '
 'stylish, making it easy to carry with you wherever you go. With a capacity '
 'of 10,000 mAh, it has enough power to charge your smartphone and other '
 'gadgets multiple times. \n'
 '\n'
 'The Powerbank 2 USB is made with a slim and sleek design, featuring smooth '
 'edges and a polished surface that is resistant to scratches. The plastic '
 'casing protects the battery from damage and gives it an elegant look. \n'
 '\n'
 'Equipped with two USB ports, this power bank can charge multiple devices '
 'simultaneously. It also comes with a USB/microUSB cable for added '
 'convenience. The LED indicator lets you know when the battery is fully '
 'charged and ready to use. \n'
 '\n'
 'This power bank is perfect for those with high-end smartphones, as it can '
 "charge them at least twice before needing to be recharged itself. It's also "
 'gr

### Issue 1: The text is too long

In [12]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 75 words.
Use at most 4 sentences.

Technical specifications: ```{fact_sheet}```
"""
response = get_completion(prompt, **prompting_params)
pprint.pprint(response)

('Introducing the Powerbank 2 USB, the perfect backup power source for your '
 'devices. With a sleek and compact design, this external battery packs a '
 'powerful punch with a capacity of 10,000 mAh. Equipped with two USB ports '
 'and a microUSB input, it can charge multiple devices at once. Plus, its thin '
 "and polished plastic casing ensures it stays scratch-free. Don't let a dead "
 'battery ruin your day, grab the Powerbank 2 USB and stay connected.')


In [13]:
words_sep = ' '
words_cnt = len(response.split(words_sep))

sentences_sep = '.'
sentences_cnt = len(response.split(sentences_sep))

print(f"{words_cnt=} {sentences_cnt=}")

words_cnt=75 sentences_cnt=6


### Issue 2: The text is too formal

In [14]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 75 words.
Use at most 4 sentences.

Use informal language only.

Technical specifications: ```{fact_sheet}```
"""
response = get_completion(prompt, **prompting_params)
pprint.pprint(response)

('Introducing the Powerbank 2 USB - the perfect backup power source for your '
 'devices! With a sleek and compact design, this external battery packs a '
 'powerful punch with a capacity of 10,000 mAh. Plus, it comes with two USB '
 'ports and a microUSB input, making it easy to charge multiple devices at '
 "once. And don't worry about scratches - the polished plastic exterior keeps "
 'it looking brand new. Get yours today and never run out of battery again!')


In [15]:
words_sep = ' '
words_cnt = len(response.split(words_sep))

sentences_sep = '.'
sentences_cnt = len(response.split(sentences_sep))

print(f"{words_cnt=} {sentences_cnt=}")

words_cnt=78 sentences_cnt=4


### The text needs some form of data table

In [16]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 150 words.
Use at most 5 sentences.

Use informal 50% informal, 50% formal language.

At the end of the description, include a table with specifications.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet}```
"""
response = get_completion(prompt, **prompting_params)
pprint.pprint(response)

('<div>\n'
 '  <h2>Powerbank 2 USB with Fast Charging</h2>\n'
 '  <p>Looking for a reliable power bank that can keep up with your busy '
 'lifestyle? Look no further than the Powerbank 2 USB with Fast Charging. This '
 'external battery pack is designed to be both stylish and compact, making it '
 "easy to take with you wherever you go. But don't let its small size fool you "
 '- with a capacity of 10,000 mAh, this power bank has plenty of energy to '
 'spare. And with two USB ports, you can easily charge multiple devices at '
 'once.</p>\n'
 '  <p>But what really sets the Powerbank 2 USB apart is its fast charging '
 "capabilities. Whether you're using it to power your smartphone, tablet, or "
 'even your router, this power bank can deliver a quick and efficient charge '
 'that will get you back up and running in no time. So why wait? Order your '
 'Powerbank 2 USB with Fast Charging today and experience the convenience of '
 'portable power for yourself!</p>\n'
 '  <table>\n'
 '    <

In [17]:
display(HTML(response))

Capacity,"10,000 mAh"
USB Ports,2
LED Indicator,Yes
Battery Type,Li-Ion
Materials,Plastic
Country of Origin,China
Miscellaneous,Scratch and dent sale (minor scratches)


In [18]:
print(datetime.datetime.now())

2023-05-01 19:33:36.453285
